This notebook is meant to explore tracking data statistics across flies. The first element in there is about generating a 'pooled' dataset where all the dataframes are concatenated in one while keeping track of flies identity and conditions.

# Libraries import

In [ ]:
import sys
from pathlib import Path
import matplotlib as mpl

mpl.rcParams["figure.figsize"] = (
    10,
    10,
)  # Change figure size including in the jupyter outputs.
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import mpmath
import cv2

sys.modules["sympy.mpmath"] = mpmath
from scipy import signal
import datetime
import dateutil
import bokeh.io
import holoviews as hv
from holoviews import opts

hv.extension(
    "bokeh",
    "matplotlib",
)
bokeh.io.output_notebook()

import panel as pn


sys.path.insert(0, "..")

from Utilities.Utils import *
from Utilities.Processing import *

import black
import jupyter_black

jupyter_black.load()

# Path and data import

In [ ]:
DataPath = [
    Path(
        "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/Optogenetics/Optobot/MultiMaze_15stepped_gated_bowtie"
    ),
    Path(
        "/Volumes/Ramdya-Lab/DURRIEU_Matthias/Experimental_data/Optogenetics/Optobot/MultiMaze_15stepped_gated_bowtie_noFood"
    ),
]

In [ ]:
# for each dataset, load the data and append it to the pooled dataframe
pooled = pd.DataFrame()
count = 0
for paths in DataPath:
    datasets = list(paths.glob("**/BallPositions_processed.feather"))
    for dataset in datasets:
        df = pd.read_feather(dataset)
        # add a column with the dataset name
        count += 1
        df["fly"] = f"Fly {count}"
        # check if the dataframe path contains "noFood"
        if "noFood" in str(dataset):
            df["food"] = "noFood"
        else:
            df["food"] = "Food"
        pooled = pooled.append(df)

# Compute average value for ysmooth grouped by trial and fly

In [ ]:
# Get average ysmooth over time grouped by trial and fly

Group = pooled.groupby(["food", "TrialNumber", "Time"])

In [ ]:
Group_avg = Group.mean()
Group_med = Group.median()

In [ ]:
Curves = (
    hv.Curve(
        data=Group_avg,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber",
            "food",
        ],
    )
    .groupby(["food", "TrialNumber"])
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Viridis"),
        tools=[
            "hover",
            "crosshair",
        ],
        muted=True,
    )
    .overlay("TrialNumber")
)
Curves

## Plot average value for ysmooth grouped by trial and fly, with a panel with and without food

In [ ]:
Curves = (
    hv.Curve(
        data=Group_avg,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber",
            "food",
        ],
    )
    .groupby(["food", "TrialNumber"])
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Viridis"),
        tools=[
            "hover",
            "crosshair",
        ],
        muted=True,
    )
    .overlay("TrialNumber")
)
Curves

## Plot with median instead to get estimates less impacted by outliers

In [ ]:
Curves = (
    hv.Curve(
        data=Group_med,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber",
            "food",
        ],
    )
    .groupby(["food", "TrialNumber"])
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Viridis"),
        tools=[
            "hover",
            "crosshair",
        ],
        muted=True,
    )
    .overlay("TrialNumber")
)
Curves

## Pooled plots

In [ ]:
Group = pooled.groupby(["TrialNumber", "Time"])
Group_avg = Group.mean()
Group_med = Group.median()

In [ ]:
Curves = (
    hv.Curve(
        data=Group_avg,
        kdims=["Time"],
        vdims=[
            "ysmooth",
            "TrialNumber",
        ],
    )
    .groupby(["TrialNumber"])
    .opts(
        height=1000,
        width=1000,
        invert_yaxis=True,
        color=hv.Palette("Viridis"),
        tools=[
            "hover",
            "crosshair",
        ],
        muted=True,
    )
    .overlay()
)
Curves

In [ ]:
hv.save(Curves, "/Users/ulric/Downloads/Average_Pooled.html")

# Plot average trial duration grouped by trial and fly

In [ ]:
Trialdurations = pooled.groupby(["fly", "TrialNumber"]).apply(lambda x: x["Time"].max())

In [ ]:
# convert Trialdurations to a dataframe and reset index to get fly and TrialNumber as columns
Trialdurations = pd.DataFrame(Trialdurations).reset_index()
# rename column 0 to "duration"
Trialdurations.rename(columns={0: "duration"}, inplace=True)

Trialdurations

In [ ]:
# Add column with converted durations in seconds
Trialdurations["duration_s"] = frame2time(
    Trialdurations["duration"], reverse=False, fps=80
)

In [ ]:
# compute average duration grouped by TrialNumber
Grouped_avg = Trialdurations.groupby("TrialNumber").mean()

In [ ]:
box = hv.BoxWhisker(data=Trialdurations, kdims="TrialNumber", vdims="duration_s").opts(
    height=500,
    width=500,
    framewise=True,
    cmap="Viridis",
    xlabel="Trial",
    ylabel="Duration (sec)",
    # ylim=(0, 40),
    # box_color="TrialNumber",
    box_fill_alpha=0,
    # invert_axes=True,
    # invert_yaxis=True,
    # box_line_color="gray",
)

points = hv.Scatter(data=Trialdurations, kdims="TrialNumber", vdims="duration_s").opts(
    framewise=True,
    cmap="Viridis",
    # invert_axes=True,
    # invert_yaxis=True,
    # ylim=(0, 40),
    color="TrialNumber",
    jitter=0.4,
)


box * points

## Grouped by food  condition

In [ ]:
Trialdurations_food = pooled.groupby(["food", "fly", "TrialNumber"]).apply(
    lambda x: x["Time"].max()
)

In [ ]:
Trialdurations_food = pd.DataFrame(Trialdurations_food).reset_index()
Trialdurations_food.rename(columns={0: "duration"}, inplace=True)

Trialdurations_food

In [ ]:
Trialdurations_food["duration_s"] = frame2time(
    Trialdurations_food["duration"], reverse=False, fps=80
)

In [ ]:
box = (
    hv.BoxWhisker(
        data=Trialdurations_food, kdims=["TrialNumber", "food"], vdims=["duration_s"]
    )
    .groupby("food")
    .opts(
        height=500,
        width=500,
        framewise=True,
        cmap="Viridis",
        xlabel="Trial",
        ylabel="Duration (sec)",
        # ylim=(0, 40),
        # box_color="TrialNumber",
        box_fill_alpha=0,
        # invert_axes=True,
        # invert_yaxis=True,
        # box_line_color="gray",
    )
)

points = (
    hv.Scatter(
        data=Trialdurations_food, kdims=["TrialNumber", "food"], vdims=["duration_s"]
    )
    .groupby("food")
    .opts(
        framewise=True,
        cmap="Viridis",
        # invert_axes=True,
        # invert_yaxis=True,
        # ylim=(0, 40),
        color="TrialNumber",
        jitter=0.4,
    )
)


box * points